In [21]:
import json
import pandas as pd
import os
from datasets import Dataset, DatasetDict

### Create proper metadata files in csv format

In [28]:
def append_entry(entries, category, audio_dir_name, is_missing):
    """Append an entry to the list of entries."""
    entry = {
        'audio_dir_name': audio_dir_name,
        'category_id': category['id'],
        'category_name': category['name'],
        'common_name': category['common_name'],
        'supercategory': category['supercategory'],
        'kingdom': category['kingdom'],
        'phylum': category['phylum'],
        'class': category['class'],
        'order': category['order'],
        'family': category['family'],
        'genus': category['genus'],
        'specific_epithet': category['specific_epithet'],
        'evaluation': category['evaluation'],
    }
    if is_missing:
        entry['file_path'] = None  # No file path for missing entries
    else:
        entry['file_path'] = os.path.join(audio_dir_name, category['file_name'])  # Add file path for valid entries
    entries.append(entry)

def convert_json_to_csv(folder: str):
    """
    Convert JSON metadata to CSV format.
    The JSON file should contain information about audio files and their categories.
    """
    path_to_dir = '/workspace/data_inatsounds'  # Update this path to your audio files directory
    path_to_json = os.path.join(path_to_dir, f'{folder}.json')
    path_to_audio_files = os.path.join(path_to_dir, folder)

    print(f"Converting JSON file {path_to_json} to CSV...")

    try:
        with open(path_to_json, 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        print(f"File {path_to_json} not found.")
        return

    rows = []
    missing_entries = []

    for category in data['categories']:
        audio_dir_name = category['audio_dir_name']
        
        audio_files_path = os.path.join(path_to_audio_files, audio_dir_name)
        
        if os.path.exists(audio_files_path):
            audio_files = [f for f in os.listdir(audio_files_path) if f.endswith('.wav')]
            
            if audio_files:
                for audio_file in audio_files:
                    category['file_name'] = audio_file
                    append_entry(rows, category, audio_dir_name, is_missing=False)
            else: 
                append_entry(missing_entries, category, audio_dir_name, is_missing=True)
        else: 
            append_entry(missing_entries, category, audio_dir_name, is_missing=True)

    valid_df = pd.DataFrame(rows)
    missing_df = pd.DataFrame(missing_entries)

    valid_df.to_csv(f'{folder}.csv', index=False)
    missing_df.to_csv(f'missing_{folder}.csv', index=False)

    total_missing = len(missing_entries)
    total_valid = len(rows)

    with open(f'{folder}_log.txt', 'w') as log_file:
        log_file.write(f"Total valid entries: {total_valid}\n")
        log_file.write(f"Total missing entries: {total_missing}\n")

    print("Metadata has been converted to metadata.csv")
    print("Missing entries have been logged in missing_metadata.csv")
    print("General information about missing entries has been logged in missing_log.txt")


In [19]:
for folder in ['train', 'test', 'val']:
    convert_json_to_csv(folder)

Converting JSON file /workspace/data_inatsounds/train.json to CSV...
Metadata has been converted to metadata.csv
Missing entries have been logged in missing_metadata.csv
General information about missing entries has been logged in missing_log.txt
Converting JSON file /workspace/data_inatsounds/test.json to CSV...
Metadata has been converted to metadata.csv
Missing entries have been logged in missing_metadata.csv
General information about missing entries has been logged in missing_log.txt
Converting JSON file /workspace/data_inatsounds/val.json to CSV...
Metadata has been converted to metadata.csv
Missing entries have been logged in missing_metadata.csv
General information about missing entries has been logged in missing_log.txt


### Create dataset

In [17]:
from datasets import Dataset, Features, Value, Audio, ClassLabel, DatasetDict
import pandas as pd
import os

In [11]:
def build_split_dataset(split_name: str, audio_dir: str, metadata_csv: str):
    df = pd.read_csv(metadata_csv)
    df = df.copy()
    df['file_path'] = df['file_path'].apply(lambda fn: os.path.join(audio_dir, fn))

    ds = Dataset.from_pandas(df, preserve_index=False)
    ds = ds.cast_column('file_path', Audio())  # default streaming mode loads array + sampling_rate
    # rename to standard 'audio' column (optional)
    ds = ds.rename_column('file_path', 'audio')
    return ds

In [12]:
splits = {
    "train": ("/workspace/data_inatsounds/train", "/workspace/data_inatsounds/train.csv"),
    "validation": ("/workspace/data_inatsounds/val", "/workspace/data_inatsounds/val.csv"),
    "test": ("/workspace/data_inatsounds/test", "/workspace/data_inatsounds/test.csv"),
}

In [18]:
dataset_dict = DatasetDict()
for split_name, (audio_dir, meta_csv) in splits.items():
    dataset_dict[split_name] = build_split_dataset(split_name, audio_dir, meta_csv)

In [19]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['audio_dir_name', 'category_id', 'category_name', 'common_name', 'supercategory', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'specific_epithet', 'evaluation', 'audio'],
        num_rows: 137012
    })
    validation: Dataset({
        features: ['audio_dir_name', 'category_id', 'category_name', 'common_name', 'supercategory', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'specific_epithet', 'evaluation', 'audio'],
        num_rows: 45698
    })
    test: Dataset({
        features: ['audio_dir_name', 'category_id', 'category_name', 'common_name', 'supercategory', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'specific_epithet', 'evaluation', 'audio'],
        num_rows: 49527
    })
})


In [20]:
print(dataset_dict["train"][0])

{'audio_dir_name': '00000_Animalia_Arthropoda_Insecta_Blattodea_Hodotermitidae_Hodotermes_mossambicus', 'category_id': 0, 'category_name': 'Hodotermes mossambicus', 'common_name': 'Northern Harvester Termite', 'supercategory': 'Insecta', 'kingdom': 'Animalia', 'phylum': 'Arthropoda', 'class': 'Insecta', 'order': 'Blattodea', 'family': 'Hodotermitidae', 'genus': 'Hodotermes', 'specific_epithet': 'mossambicus', 'evaluation': 0, 'audio': {'path': '/workspace/data_inatsounds/train/00000_Animalia_Arthropoda_Insecta_Blattodea_Hodotermitidae_Hodotermes_mossambicus/646794f4-4945-4401-98f0-580105f7fa48.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 22050}}


In [23]:
# dataset_dict.save_to_disk('/workspace')

In [25]:
len(dataset_dict["train"]), len(dataset_dict["test"]), len(dataset_dict["validation"])

(137012, 49527, 45698)

### Upload dataset to huggingface

In [ ]:
from huggingface_hub import login
login("insert-code-here")

In [28]:
dataset_dict.push_to_hub("DBD-research-group/inat_sounds", private=True)  # set private=False to make it public

Uploading the dataset shards:   0%|          | 0/255 [00:00<?, ?it/s]

Map:   0%|          | 0/538 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Map:   0%|          | 0/538 [00:00<?, ? examples/s]

...

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Map:   0%|          | 0/526 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


HfHubHTTPError: (Request ID: Root=1-68b725c7-7273290f16b71f952b6b0902;d001cd61-3457-4fcd-84a0-7f2975e8d278)

403 Forbidden: Private repository storage limit reached, please upgrade your plan to increase your private storage limit.
Cannot access content at: https://huggingface.co/api/datasets/DBD-research-group/inat_sounds/commit/main.
Make sure your token has the correct permissions.